In [ ]:
%run ../../_pre_run.ipynb

# Missing Value Handling


## Table orders

Let’s examine missing values.

In [ ]:
tmp_miss = df_orders.explore.detect_anomalies(return_mode='by_column')

To calculate time intervals between different dates, we should ideally fill missing values between dates if the next stage’s date is present.

We’ll replace missing timestamps.

**order_delivered_carrier_dt**

Let’s look at orders missing the carrier handoff date but having a delivery date or delivery status.

In [ ]:
tmp_miss['order_delivered_carrier_dt'].order_status.value_counts()

In [ ]:
tmp_mask = lambda x: (x.order_delivered_carrier_dt.isna()) & (x.order_delivered_customer_dt.notna() | (x.order_status == 'Delivered'))

In [ ]:
df_orders[tmp_mask].head()

There are only 2 such orders—we’ll replace them with the median value.

In [ ]:
order_carrier_time = (df_orders.order_delivered_carrier_dt - df_orders.order_approved_dt).median()
order_carrier_time

In [ ]:
df_orders.loc[tmp_mask, 'order_delivered_carrier_dt'] = df_orders.order_approved_dt + order_carrier_time

**order_approved_dt**

Let’s examine orders missing the payment approval date but having a carrier handoff timestamp.

In [ ]:
tmp_miss['order_approved_dt'].order_status.value_counts()

In [ ]:
tmp_mask = lambda x: x.order_approved_dt.isna() & x.order_delivered_carrier_dt.notna()

In [ ]:
print(f'Rows count: {df_orders[tmp_mask].shape[0]}')
df_orders[tmp_mask].head()

Since there are only 14 such orders, we’ll simply fill the gaps using this time.

Let’s check the median time from order creation to payment approval.

In [ ]:
order_processing_time_hours = (df_orders.order_approved_dt - df_orders.order_purchase_dt).median()
order_processing_time_hours

In [ ]:
df_orders.loc[tmp_mask, 'order_approved_dt'] = df_orders.order_purchase_dt + order_processing_time_hours

**order_delivered_customer_dt**

Let’s examine delivered orders missing the customer receipt date.

In [ ]:
tmp_miss['order_delivered_customer_dt'].order_status.value_counts()

In [ ]:
tmp_miss['order_delivered_customer_dt'][lambda x: x.order_status == 'Delivered']

There are only 8 such orders—we’ll also replace them with the median value.

In [ ]:
order_customer_time = (df_orders.order_delivered_customer_dt - df_orders.order_delivered_carrier_dt).median()
order_customer_time

In [ ]:
tmp_mask = df_orders.order_delivered_customer_dt.isna() & (df_orders.order_status=='Delivered')

In [ ]:
df_orders.loc[tmp_mask, 'order_delivered_customer_dt'] = df_orders.order_delivered_carrier_dt + order_customer_time

Missing values in review_comment_title and review_comment_message indicate these fields were left blank, so processing them is unnecessary.

## Table df_products

Let’s examine missing values in df_products.

In [ ]:
tmp_miss = df_products.explore.detect_anomalies(return_mode='by_column')

In product_category_name, we’ll replace missing values with 'Missing in Products'.

In [ ]:
df_products['product_category_name'] = df_products['product_category_name'].cat.add_categories(['Missing in Products'])
df_products['product_category_name'] = df_products['product_category_name'].fillna('Missing in Products')

Since only 2 orders lack specifications, we’ll replace them with the median value for their product category.

Let’s check group sizes if we replace within these groups.

In [ ]:
category_columns = 'product_category_name'
df_products['product_weight_g'].preproc.check_group_counts(category_columns=category_columns)

The group sizes are sufficient, so we’ll replace with the group median.

In [ ]:
df_products[df_products[['product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']].isna().any(axis=1)]

In [ ]:
df_products['product_weight_g'].preproc.fill_missing_by_category(category_columns=category_columns, inplace=True)
df_products['product_length_cm'].preproc.fill_missing_by_category(category_columns=category_columns, inplace=True)
df_products['product_height_cm'].preproc.fill_missing_by_category(category_columns=category_columns, inplace=True)
df_products['product_width_cm'].preproc.fill_missing_by_category(category_columns=category_columns, inplace=True)

Missing values in product name length and product description length won’t be filled, as we won’t use them for analysis.

Missing values in photo count will be replaced with 1, as it’s both the median and mode.

In [ ]:
df_products.loc[df_products.product_photos_qty.isna(), 'product_photos_qty'] = 1

Let’s check how many missing values remain.


In [ ]:
df_products.explore.detect_anomalies()

In [ ]:
del tmp_miss

In [ ]:
%run ../../_post_run.ipynb